In [4]:
# Import required libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations
from typing import Tuple, List, Union  # For type hints
from datetime import date  # For date handling

def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare the input DataFrame by calculating returns and extracting dates.
    This function handles the initial data preprocessing steps.
    
    Args:
        df: DataFrame with 'datetime' and 'close' columns
        
    Returns:
        DataFrame with added 'return' and 'date' columns, sorted by datetime
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Convert datetime column to pandas datetime format
    df['datetime'] = pd.to_datetime(df['datetime'])
    
    # Sort data chronologically to ensure correct return calculations
    df.sort_values('datetime', inplace=True)
    
    # Calculate percentage returns (price changes)
    # pct_change() computes the percentage change between consecutive elements
    df['return'] = df['close'].pct_change()
    
    # Extract date component from datetime for daily grouping
    df['date'] = df['datetime'].dt.date
    
    # Remove rows with NaN returns (first row will have NaN as there's no previous price)
    return df.dropna(subset=['return'])

def get_window_dates(df: pd.DataFrame, specific_date: Union[str, date], 
                     window_size: int) -> List[date]:
    """
    Get the dates for the rolling window analysis.
    This function ensures we have enough data for the analysis and returns
    the appropriate date range.
    
    Args:
        df: Prepared DataFrame with 'date' column
        specific_date: Target date for analysis
        window_size: Number of days in rolling window
        
    Returns:
        List of dates in the rolling window
        
    Raises:
        ValueError: If specific_date not found or insufficient data
    """
    # Convert specific_date to datetime.date object if it's a string
    specific_date = pd.to_datetime(specific_date).date()
    
    # Get unique dates and sort them chronologically
    unique_dates = sorted(df['date'].unique())
    
    # Validate that the specific date exists in our data
    if specific_date not in unique_dates:
        raise ValueError("Specific date not found in the data.")
    
    # Find the index of our target date
    date_index = unique_dates.index(specific_date)
    
    # Check if we have enough historical data for the window
    if date_index < window_size - 1:
        raise ValueError("Not enough data for the specified window size.")
    
    # Return the window of dates (including the specific date)
    return unique_dates[date_index - window_size + 1 : date_index + 1]

def calculate_daily_probabilities(window_data: pd.DataFrame, 
                                num_bins: int) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    Calculate probability distributions for each day in the window.
    This function creates histograms of returns and converts them to probability distributions.
    
    Args:
        window_data: DataFrame containing window data with 'return' and 'date' columns
        num_bins: Number of bins for histogram (determines granularity of distribution)
        
    Returns:
        Tuple containing:
        - DataFrame of daily probability distributions
        - Array of bin edges used for the histograms
    """
    # Find the range of returns in our window
    min_return = window_data['return'].min()
    max_return = window_data['return'].max()
    
    # Create evenly spaced bins across the return range
    bins = np.linspace(min_return, max_return, num_bins + 1)
    
    # Calculate probability distribution for each day
    daily_probs = []
    for date in window_data['date'].unique():
        # Get returns for this specific day
        daily_returns = window_data[window_data['date'] == date]['return']
        
        # Create histogram of returns
        counts, _ = np.histogram(daily_returns, bins=bins)
        
        # Convert counts to probabilities
        # If there are no returns (counts.sum() == 0), use zeros
        probabilities = counts / counts.sum() if counts.sum() > 0 else np.zeros(num_bins)
        daily_probs.append(probabilities)
    
    # Create DataFrame with dates as index and probabilities as columns
    return pd.DataFrame(daily_probs, index=window_data['date'].unique()), bins

def calculate_risk(df: pd.DataFrame, window_dates: List[date]) -> float:
    """
    Calculate risk as standard deviation of daily returns.
    This is a traditional measure of volatility/risk in financial markets.
    
    Args:
        df: Original DataFrame with price data
        window_dates: List of dates in the analysis window
        
    Returns:
        Risk metric (standard deviation of daily returns)
    """
    # Get the last price of each day
    daily_close = df.groupby('date')['close'].last()
    
    # Calculate daily returns
    daily_returns = daily_close.pct_change().dropna()
    
    # Filter returns for our window
    window_daily_returns = daily_returns.loc[window_dates]
    
    # Calculate standard deviation of daily returns
    return window_daily_returns.std()

def calculate_ambiguity_and_risk(df: pd.DataFrame, 
                               specific_date: Union[str, date],
                               window_size: int = 5, 
                               num_bins: int = 20) -> Tuple[float, pd.Series, float]:
    """
    Main function to calculate ambiguity and risk metrics for a specific date.
    This function orchestrates the entire analysis process.
    
    Args:
        df: DataFrame with 'datetime' and 'close' columns
        specific_date: Target date for analysis
        window_size: Number of days in rolling window (default: 5)
        num_bins: Number of bins for histogram (default: 20)
        
    Returns:
        Tuple containing:
        - ambiguity_metric: Average standard deviation across bins
        - interval_std: Standard deviations for each bin
        - risk: Standard deviation of daily returns
    """
    # Step 1: Prepare the data
    df = prepare_data(df)
    
    # Step 2: Get the dates for our analysis window
    window_dates = get_window_dates(df, specific_date, window_size)
    
    # Step 3: Filter data for our window
    window_data = df[df['date'].isin(window_dates)].copy()
    
    # Step 4: Calculate probability distributions
    prob_df, _ = calculate_daily_probabilities(window_data, num_bins)
    
    # Step 5: Calculate ambiguity metrics
    # Standard deviation across days for each bin
    interval_std = prob_df.std(axis=0)
    # Average standard deviation across all bins
    daily_ambiguity_metric = interval_std.mean()
    
    # Step 6: Calculate risk
    risk = calculate_risk(df, window_dates)
    
    return daily_ambiguity_metric, interval_std, risk

def load_and_prepare_data(file_path):
    # Read the CSV with the expected columns
    df = pd.read_csv(file_path, usecols=['datetime_nano', 'SSE.000300.close'])
    df.rename(columns={'SSE.000300.close': 'close'}, inplace=True)
    
    # Convert 'datetime_nano' to a datetime object.
    # First interpret as UTC date, then convert to local time (Asia/Shanghai), then floor to minute.
    df['datetime'] = (pd.to_datetime(df['datetime_nano'], utc=True)
                        .dt.tz_convert('Asia/Shanghai')
                        .dt.floor('min')
                        .dt.tz_localize(None))
    df.drop('datetime_nano', axis=1, inplace=True)
    
    # Sort by datetime
    df.sort_values('datetime', inplace=True)
    
    return df

    
# Example usage
if __name__ == "__main__":
    # Example of how to use the code
    # Note: You need to have a DataFrame 'df' with 'datetime' and 'close' columns
    file_path = '../../data/SSE.000300.csv'  # Ensure the path is correct
    df = load_and_prepare_data(file_path)
    # Set parameters
    specific_date = '2018-01-09'
    window_size = 5  # 5-day window
    num_bins = 50    # 20 bins for histogram
    
    # Calculate metrics
    ambiguity_metric, interval_std, risk = calculate_ambiguity_and_risk(
        df, 
        specific_date, 
        window_size=window_size, 
        num_bins=num_bins
    )
    
    # Print results
    print(f"Ambiguity Metric on {specific_date}: {ambiguity_metric}")
    print(f"Risk (Standard Deviation of Daily Returns) on {specific_date}: {risk}")
    #print("Standard deviation across bins:")
    #print(interval_std)

Ambiguity Metric on 2018-01-09: 0.005197856487538408
Risk (Standard Deviation of Daily Returns) on 2018-01-09: 0.0017383918259038763
